This notebook provides a benchmarking of the surrogate + SAEM implementation on a well-known data set, the **theophylline** data.

The study was reported by Upton and analyzed by Boeckmann et al. (1994).


In [ ]:
import numpy as np
import pandas as pd 
import torch
from plotnine import *
import pickle

%load_ext autoreload
%autoreload 2

import vpop_calibration



In [ ]:
# Setup the training data for the surrogate model
# An analytical expression is available for the model


def analytical_model(d, v, ka, cl, t):
    """Analytical expression of a 1 compartment PK model

    Args:
        t: time in h
        d: Dose in mg
        v: Distribution volume in mL
        ka: Absorption rate constant in mL/h
        cl: Clearance rate constant in mL/h

    Returns:
        y: Predicted concentration
    """
    ke = cl / v
    y = d * ka / (v * (ka - ke)) * (torch.exp(-ke * t) - torch.exp(-ka * t))
    return y


struct_model = vpop_calibration.StructuralAnalytical(
    analytical_model, ["d", "v", "ka", "cl"], ["concentration"]
)

In [ ]:
theophylline_data_url = "https://monolixsuite.slp-software.com/__attachments/49742071/theophylline_data.txt?inst-v=c9fc1b05-711a-4d91-875c-0e64dc343911"

df = pd.read_csv(theophylline_data_url, sep="\t")

display(df.head())

patients_df = df.loc[df["AMT"] != "."].copy()
patients_df.loc[:, "d"] = patients_df.apply(
    lambda r: float(r["AMT"]) * r["WEIGHT"], axis=1
)
patients_df = patients_df.rename(columns={"ID": "id", "WEIGHT": "bw"})[
    ["id", "d", "bw"]
]
display(patients_df.head())

obs_df = (
    df.loc[df["AMT"] == "."]
    .copy()
    .rename(columns={"ID": "id", "CONC": "value", "TIME": "time"})[
        ["id", "time", "value"]
    ]
    .astype({"value": "float"})
)
obs_df["output_name"] = "concentration"
obs_df["protocol_arm"] = "identity"
display(obs_df.head())

In [ ]:
init_log_mi = {}
init_log_pdu = {
    "ka": {"mean": 0.0, "sd": 0.05},
    "cl": {"mean": -1.0, "sd": 0.05},
    "v": {"mean": 1.0, "sd": 0.05},
}
covariate_map = {"ka": {}, "v": {}, "cl": {"bw": {"coef": "cov_bw_cl", "value": 0.01}}}
init_res_var = [0.01]
nlme_model = vpop_calibration.NlmeModel(
    structural_model=struct_model,
    patients_df=patients_df,
    init_log_MI=init_log_mi,
    init_PDU=init_log_pdu,
    covariate_map=covariate_map,
    init_res_var=init_res_var,
    error_model_type="additive",
)

optimizer = vpop_calibration.PySaem(
    model=nlme_model,
    observations_df=obs_df,
    mcmc_first_burn_in=100,
    mcmc_nb_transitions=1,
    nb_phase1_iterations=500,
    nb_phase2_iterations=1000,
    plot_frames=500,
)

In [ ]:
optimizer.run()

In [ ]:
print(nlme_model.PDU_names)
print(
    [
        torch.exp(
            nlme_model.population_betas[nlme_model.population_betas_names.index(name)]
        ).item()
        for name in nlme_model.PDU_names
    ]
)

In [ ]:
vpop_calibration.plot_map_estimates(nlme_model)